---
title: Core
subtitle: Core data structures and algorithms
---

In [2]:
#| default_exp core

## Variables

In [3]:
#| export
from pydantic import BaseModel
import polars as pl
from datetime import datetime
from functools import cached_property
from abc import abstractmethod

In [10]:
# | export
class Variable(BaseModel):
    name: str = None
    description: str = None
    unit: str = None

    timerange: list[datetime] = None

    # similar to `speasy`
    provider: str = "cda"
    dataset: str = None
    parameter: str = None
    product: str = None
    """product name should be unique"""

    def to_polars(self) -> pl.LazyFrame:
        pass

    def preview(self):
        return self.to_polars().head().collect()

    @abstractmethod
    @cached_property
    def data(self):
        """Retrieve the data if not already done."""
        ...

    def dump(self, path: str):
        """Dump the configuration to a file."""
        import yaml

        with open(path, "w") as f:
            yy = yaml.load(
                self.model_dump_json(exclude_defaults=True), Loader=yaml.FullLoader
            )
            yaml.dump(yy, f)
    
    @classmethod
    def load_from_file(cls, path: str):
        """Load the configuration from a file."""
        import yaml

        with open(path, "r") as f:
            yy = yaml.load(f, Loader=yaml.FullLoader)
            return cls(**yy)


class Variables(Variable):
    class Config:
        arbitrary_types_allowed = True  # RuntimeError: no validator found for <class 'speasy.products.variable.SpeasyVariable'>, see  `arbitrary_types_allowed` in Config

    products: list[str] = None
    parameters: list[str] = None

    _data: list[Variable] = None

In [20]:
# test
vs = Variables(products=['file'])
vs.dump("test.yaml")
Variables.load_from_file("test.yaml") == vs

True

In [2]:
#| hide
import nbdev
nbdev.nbdev_export()